# Basic Deep Learning

## Forward Propagation

In [ ]:
from IPython.display import Image
import numpy as np

In [ ]:
Image('https://s3.amazonaws.com/assets.datacamp.com/production/course_3524/datasets/1_4.png')

In [ ]:
input_data = np.array([3,5])
weights = {'node_0': np.array([2, 4]), 'node_1': np.array([ 4, -5]), 'output': np.array([2, 7])}

In [ ]:
# Imitage dot products with element-wise multiplication & sum
node_0_value = (input_data * weights['node_0']).sum() # matrix form input_data@weights['node_0']
node_1_value = (input_data * weights['node_1']).sum() # matrix form input_data@weights['node_1']

In [ ]:
hidden_layer_outputs = np.array([node_0_value, node_1_value])
output = (hidden_layer_outputs * weights['output']).sum()
print(output)

## Activation Fn  
capture Linear / Non-linear relationship in each node  
`ReLu (Rectify Linear activation)` is the most efficient & used

In [ ]:
def relu(input):
    return max(input, 0)

In [ ]:
inputs = np.array([3, 2])
weights = {'node_0':np.array([2,1]), 'node_1':np.array([0, 0]), 'output':np.array([2,2])}

node_0_value = (inputs * weights['node_0']).sum()
node_0_output = relu(node_0_value)

node_1_value = (inputs * weights['node_1']).sum() 
node_1_output = relu(node_1_value)

hidden_layer_data = np.array([node_0_output, node_1_output])

hidden_layer_value = (hidden_layer_data * weights['output']).sum()
# hidden_layer_output = relu(hidden_layer_value)
hidden_layer_output = hidden_layer_value

print(hidden_layer_output)
print(hidden_layer_output-5)

## Optimization the error

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error

In [ ]:
inputs = np.array([0, 3])
target_actual = 3

weights = {'node_0':np.array([2,1]), 'node_1':np.array([1, 2]), 'output':np.array([1,1])}

node_0_value = (inputs * weights['node_0']).sum()
node_0_output = relu(node_0_value)

node_1_value = (inputs * weights['node_1']).sum() 
node_1_output = relu(node_1_value)

hidden_layer_data = np.array([node_0_output, node_1_output])

hidden_layer_value = (hidden_layer_data * weights['output']).sum()
# hidden_layer_output = relu(hidden_layer_value)
hidden_layer_output = hidden_layer_value

print(f'Target : Predict={hidden_layer_output}, Actual={target_actual}, error={hidden_layer_output-target_actual}')

## Gradient Descend

## Backpropagation

# Keras

In [84]:
import pandas as pd

import keras
from keras.layers import Dense
from keras.models import Sequential, load_model
from keras.utils import to_categorical
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping

In [70]:
df = \
(pd
 .read_csv('../input/titanic/train.csv')
 .drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
 .rename(columns=str.lower)
 .assign(male = lambda x : pd.get_dummies(x['sex'], drop_first=True))
 .drop('sex', axis=1)
 .dropna()
 .reset_index(drop=True)
)

emb_encode = pd.get_dummies(df['embarked'], prefix='embark')

df = pd.concat([df.drop('embarked', axis=1), emb_encode], axis=1)

In [71]:
df.head()

,survived,pclass,age,sibsp,parch,fare,male,embark_C,embark_Q,embark_S
0,0,3,22.0,1,0,7.2500,1,0,0,1
1,1,1,38.0,1,0,71.2833,0,1,0,0
2,1,3,26.0,0,0,7.9250,0,0,0,1
3,1,1,35.0,1,0,53.1000,0,0,0,1
4,0,3,35.0,0,0,8.0500,1,0,0,1


In [72]:
df.isna().sum()

survived    0
pclass      0
age         0
sibsp       0
parch       0
fare        0
male        0
embark_C    0
embark_Q    0
embark_S    0
dtype: int64

In [73]:
target = to_categorical(df['survived'])
predictors = df.drop('survived', axis=1).to_numpy()

In [74]:
n_cols = predictors.shape[1]

model = Sequential()
model.add(Dense(32, activation='relu', input_shape=(n_cols,)))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(predictors, target, epochs=10)

Epoch 1/10
23/23 [==============================] - 0s 1ms/step - loss: 3.7080 - accuracy: 0.5574
Epoch 2/10
23/23 [==============================] - 0s 1ms/step - loss: 0.8236 - accuracy: 0.6342
Epoch 3/10
23/23 [==============================] - 0s 1ms/step - loss: 0.8945 - accuracy: 0.6849
Epoch 4/10
23/23 [==============================] - 0s 1ms/step - loss: 0.6615 - accuracy: 0.6686
Epoch 5/10
23/23 [==============================] - 0s 1ms/step - loss: 0.6457 - accuracy: 0.6697
Epoch 6/10
23/23 [==============================] - 0s 1ms/step - loss: 0.5940 - accuracy: 0.7205
Epoch 7/10
23/23 [==============================] - 0s 1ms/step - loss: 0.6186 - accuracy: 0.6732
Epoch 8/10
23/23 [==============================] - 0s 1ms/step - loss: 0.5898 - accuracy: 0.6990
Epoch 9/10
23/23 [==============================] - 0s 1ms/step - loss: 0.6555 - accuracy: 0.6454
Epoch 10/10
23/23 [==============================] - 0s 1ms/step - loss: 0.5918 - accuracy: 0.7144


In [60]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 32)                320       
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 66        
Total params: 386
Trainable params: 386
Non-trainable params: 0
_________________________________________________________________


In [61]:
model.save('titanic_keras_sgd.h5')

In [62]:
model = load_model('titanic_keras_sgd.h5')

In [63]:
model.predict(predictors)

array([[0.85329556, 0.1467044 ],
       [0.36334983, 0.63665015],
       [0.8536829 , 0.14631714],
       ...,
       [0.45185217, 0.5481478 ],
       [0.5074541 , 0.49254596],
       [0.8951586 , 0.10484146]], dtype=float32)

In [66]:
predict_prob_true = model.predict(predictors)[:,1]

In [67]:
predict_prob_true

array([0.1467044 , 0.63665015, 0.14631714, 0.57277817, 0.07680956,
       0.36477587, 0.4363523 , 0.13846324, 0.55326533, 0.47197378,
       0.11934554, 0.17322321, 0.22133343, 0.2852464 , 0.06151647,
       0.4097411 , 0.19528764, 0.31806064, 0.13247266, 0.29929247,
       0.50919527, 0.4411567 , 0.24320592, 0.8970066 , 0.2977688 ,
       0.01863759, 0.6848079 , 0.50832367, 0.16308774, 0.41533646,
       0.4103554 , 0.07735638, 0.3731331 , 0.5537116 , 0.24043891,
       0.42966998, 0.46009192, 0.16057724, 0.61736846, 0.38553017,
       0.32761577, 0.2614563 , 0.11563464, 0.49537376, 0.44050837,
       0.16298707, 0.6440201 , 0.43392786, 0.17618144, 0.18559858,
       0.2350274 , 0.13059966, 0.12788208, 0.4515972 , 0.65812206,
       0.19225857, 0.58171564, 0.12751932, 0.5085093 , 0.15452075,
       0.16358417, 0.11492053, 0.5700172 , 0.32478106, 0.16234863,
       0.46967545, 0.90074736, 0.13758087, 0.1059069 , 0.17090024,
       0.5390825 , 0.30374256, 0.01571602, 0.08864559, 0.64158

# Optimization

In [68]:
def get_new_model(predictors):
    n_cols = predictors.shape[1]
    model = Sequential()
    model.add(Dense(32, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(2, activation='softmax'))
    return model

In [80]:
lr_list = [0.00001,0.01, 1]

for lr in lr_list:
    opt = SGD(learning_rate=lr)
    model = get_new_model(predictors)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(predictors, target)

23/23 [==============================] - 0s 1ms/step - loss: 435.7632 - accuracy: 0.5766


In [83]:
model = get_new_model(predictors)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(predictors, target, validation_split=0.3)

16/16 [==============================] - 1s 24ms/step - loss: 7.9863 - accuracy: 0.5343 - val_loss: 7.4518 - val_accuracy: 0.2944


In [85]:
early_stop = EarlyStopping(patience=2)
model = get_new_model(predictors)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(predictors, target, validation_batch_size=0.3, epochs=30, callbacks=[early_stop])

Epoch 1/30
23/23 [==============================] - 0s 1ms/step - loss: 3.8416 - accuracy: 0.3076
Epoch 2/30
23/23 [==============================] - 0s 1ms/step - loss: 0.7278 - accuracy: 0.6036
Epoch 3/30
23/23 [==============================] - 0s 1ms/step - loss: 0.6878 - accuracy: 0.6772
Epoch 4/30
23/23 [==============================] - 0s 1ms/step - loss: 0.6124 - accuracy: 0.6847
Epoch 5/30
23/23 [==============================] - 0s 1ms/step - loss: 0.6176 - accuracy: 0.6753
Epoch 6/30
23/23 [==============================] - 0s 1ms/step - loss: 0.6014 - accuracy: 0.6997
Epoch 7/30
23/23 [==============================] - 0s 1ms/step - loss: 0.5685 - accuracy: 0.7208
Epoch 8/30
23/23 [==============================] - 0s 1ms/step - loss: 0.5678 - accuracy: 0.7260
Epoch 9/30
23/23 [==============================] - 0s 1ms/step - loss: 0.5768 - accuracy: 0.7022
Epoch 10/30
23/23 [==============================] - 0s 1ms/step - loss: 0.6087 - accuracy: 0.6669
Epoch 11/30
23/23 [